In [ ]:
!pip install gensim


In [ ]:
!pip install docx2txt
!pip install pdfplumber
!pip install PyMuPDF
!pip install datasets

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlo

In [ ]:
import pandas as pd
import fitz
import ast
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import docx2txt
import pdfplumber

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# dataset : https://huggingface.co/datasets/batuhanmtl/job-skill-set

# Dataset preprocessing

In [ ]:
from datasets import load_dataset

ds = load_dataset("batuhanmtl/job-skill-set")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1167 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("batuhanmtl/job-skill-set")
df = pd.DataFrame(ds["train"])
print(df.head())
print("\nMissing values:\n", df.isnull().sum())


       job_id category                                 job_title  \
0  3902668440       HR              Sr Human Resource Generalist   
1  3905823748       HR                   Human Resources Manager   
2  3905854799       HR               Director of Human Resources   
3  3905834061       HR             Chief Human Resources Officer   
4  3906250451       HR  Human Resources Generalist (Hybrid Role)   

                                     job_description  \
0  SUMMARY\nTHE SR. HR GENERALIST PROVIDES HR EXP...   
1  BE PART OF A STELLAR TEAM AT YSB AS THE MANAGE...   
2  OUR CLIENT IS A THRIVING ORGANIZATION OFFERING...   
3  JOB TITLE: CHIEF HUMAN RESOURCES OFFICER (CHRO...   
4  DESCRIPTION\n\n WHO WE ARE \n\nAVI-SPL IS A DI...   

                                       job_skill_set  
0  ['employee relations', 'talent acquisition', '...  
1  ['Talent Acquisition', 'Employee Performance M...  
2  ['Human Resources Management', 'Recruitment', ...  
3  ['talent management', 'organiza

In [ ]:
ds = load_dataset("batuhanmtl/job-skill-set")

# Convert to Pandas DataFrame
df = pd.DataFrame(ds["train"])

In [ ]:
df.columns = df.columns.str.lower().str.replace(" ", "_")


In [ ]:
df["job_skill_set"] = df["job_skill_set"].fillna("no skills mentioned")
df.dropna(subset=["job_title", "job_description"], inplace=True)


In [ ]:
def convert_to_list(skill_string):
    if isinstance(skill_string, str):
        try:
            return ast.literal_eval(skill_string)
        except (SyntaxError, ValueError):
            return ["no skills mentioned"]
    return skill_string


In [ ]:
df["job_skill_set"] = df["job_skill_set"].apply(convert_to_list)
df["category"] = df["category"].str.lower()

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


In [ ]:
def clean_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = " ".join(text.split())  # Remove extra spaces
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatization & Stopword removal
    return text


In [ ]:
df["job_description"] = df["job_description"].apply(clean_text)


In [ ]:
df["job_skill_set"] = df["job_skill_set"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.to_csv("cleaned_job_dataset.csv", index=False)
print("Preprocessing complete! Saved as 'cleaned_job_dataset.csv'.")
print(df.head())

Preprocessing complete! Saved as 'cleaned_job_dataset.csv'.
       job_id category                                 job_title  \
0  3902668440       hr              Sr Human Resource Generalist   
1  3905823748       hr                   Human Resources Manager   
2  3905854799       hr               Director of Human Resources   
3  3905834061       hr             Chief Human Resources Officer   
4  3906250451       hr  Human Resources Generalist (Hybrid Role)   

                                     job_description  \
0  summary sr hr generalist provides hr expertise...   
1  part stellar team ysb manager human resource j...   
2  client thriving organization offering unique o...   
3  job title chief human resource officer chroind...   
4  description avispl digital enablement solution...   

                                       job_skill_set  
0  employee relations, talent acquisition, perfor...  
1  Talent Acquisition, Employee Performance Manag...  
2  Human Resources Management

# Resume preprocessing

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + " "  # Extract text from each page
    return text.strip()

In [ ]:
def fix_spacing(text):
    """Fix missing spaces between words using regex."""
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)  # Add space between camel case words
    text = re.sub(r'(?<=[a-zA-Z])(?=\d)', ' ', text)  # Add space between words and numbers
    text = re.sub(r'(?<=\d)(?=[a-zA-Z])', ' ', text)  # Add space between numbers and words
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)  # Add space between lowercase & uppercase transitions
    return text


In [ ]:
def preprocess_resume(text):
    """Preprocess resume text: Fix spacing, remove unwanted characters, lemmatize, and remove stopwords."""
    text = fix_spacing(text)  # Fix missing spaces
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = word_tokenize(text)  # Tokenize words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization & stopword removal
    return " ".join(words)


In [ ]:
pdf_path = "/content/Business Administration Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)


In [ ]:
cleaned_resume = preprocess_resume(resume_text)


In [ ]:
print(" Cleaned Resume Text:")
print(cleaned_resume)

 Cleaned Resume Text:
department career development erwin 116 585 2455721 division student campus life wwwgeneseoeducareerdevelopment pg 1 business administration resume template name address email phone linked profile education state university new york geneseo geneseo ny expected may 2019 school business nyaacsb accreditation bachelor science business administration gpa 364 relevant experience stone consulting firm rochester ny august 2018 present investment analyst provide advisory service company organization guide making decision merger acquisition stock bond financial opportunity assess business strategy client including analysis working capital process workflow financial inefficiency new business opportunity manage model analyze best worst case scenario used senior banker negotiate 5 10 lower price client well miller kern company new york ny june 2018 august 2018 senior investment advisor intern sustained business unit function defining goal exploring growth opportunity construc

# why concatenation is needed

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv("/content/cleaned_job_dataset.csv")
label_encoder = LabelEncoder()
df["category_encoded"] = label_encoder.fit_transform(df["category"])
text_features = ["job_title", "job_description", "job_skill_set"]


mi_scores = {}
vectorizers = {}

for feature in text_features:
    # Convert text to numerical form using TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)  # Limit features for efficiency
    X_tfidf = vectorizer.fit_transform(df[feature].astype(str))  # Convert to string
    mi_score = mutual_info_classif(X_tfidf.toarray(), df["category_encoded"], discrete_features=False)

    mi_scores[feature] = np.mean(mi_score)
    vectorizers[feature] = vectorizer


print("Mutual Information Scores:")
for feature, score in mi_scores.items():
    print(f"   - {feature}: {score:.5f}")


best_feature = max(mi_scores, key=mi_scores.get)
print(f"\n Best feature for embeddings: {best_feature}")


best_vectorizer = vectorizers[best_feature]
X_selected_feature = best_vectorizer.fit_transform(df[best_feature].astype(str))


df_tfidf = pd.DataFrame(X_selected_feature.toarray(), columns=best_vectorizer.get_feature_names_out())


df_tfidf.to_csv("selected_feature_embeddings.csv", index=False)

print("\n Feature selection complete! Using this column for further processing.")


#concatenation code

In [ ]:
import pandas as pd

def preprocess_text(text):
    """Basic text preprocessing: lowercase, remove extra spaces."""
    if isinstance(text, str):
        return ' '.join(text.lower().strip().split())
    return ''
df = pd.read_csv("/content/cleaned_job_dataset.csv")

df['job_title_cleaned'] = df['job_title'].apply(preprocess_text)
df['job_description_cleaned'] = df['job_description'].apply(preprocess_text)
df['job_skill_set_cleaned'] = df['job_skill_set'].apply(preprocess_text)


df['combined_text'] = (
    df['job_title_cleaned'] + " " +
    df['job_description_cleaned'] + " " +
    df['job_skill_set_cleaned']
)
df.to_csv("/content/final_combined_jobs.csv", index=False)

print(df[['job_title', 'combined_text']].head())

In [ ]:
data=pd.read_csv("/content/final_combined_jobs.csv")
print(data.head(4))

#tf idf and sbert + cosine embeddings

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:

df = pd.read_csv("/content/final_combined_jobs.csv")
corpus = df['combined_text'].astype(str).tolist()

resume_text = cleaned_resume


In [ ]:
def tfidf_embedding(corpus, resume_text):
    vectorizer = TfidfVectorizer(max_features=5000)
    job_embeddings = vectorizer.fit_transform(corpus).toarray()
    resume_embedding = vectorizer.transform([resume_text]).toarray()
    return job_embeddings, resume_embedding


In [ ]:
def sbert_embedding(corpus, resume_text):
    model = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight SBERT model
    job_embeddings = model.encode(corpus)
    resume_embedding = model.encode([resume_text])
    return job_embeddings, resume_embedding

In [ ]:
def compute_similarity(job_embeddings, resume_embedding):
    similarities = cosine_similarity(job_embeddings, resume_embedding)
    return similarities.flatten()


In [ ]:
job_tfidf, resume_tfidf = tfidf_embedding(corpus, resume_text)
tfidf_scores = compute_similarity(job_tfidf, resume_tfidf)

In [ ]:
job_sbert, resume_sbert = sbert_embedding(corpus, resume_text)
sbert_scores = compute_similarity(job_sbert, resume_sbert)

In [ ]:
df['tfidf_score'] = tfidf_scores
df['sbert_score'] = sbert_scores

In [ ]:
df_sorted = df.sort_values(by="sbert_score", ascending=False)

In [ ]:
print("\n**Top 5 Job Recommendations (SBERT Similarity)**")
print(df_sorted[['job_title', 'sbert_score']].head(5))

In [ ]:
print("\n **Top 5 Job Recommendations (TF-IDF Similarity)**")
print(df.sort_values(by="tfidf_score", ascending=False)[['job_title', 'tfidf_score']].head(5))

# Embeddings: Sentence-BERT  
# Neural Network: Multi-Layer Perceptron

In [ ]:
!pip install sentence-transformers
!pip install tensorflow
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df = pd.read_csv("/content/final_combined_jobs.csv")

In [ ]:
df['original_job_id'] = df['job_id']
df.reset_index(drop=True, inplace=True)

In [ ]:
print("Generating embeddings for job listings...")
model_st = SentenceTransformer('all-MiniLM-L6-v2')
job_embeddings = model_st.encode(df['combined_text'].tolist(), show_progress_bar=True)


Generating embeddings for job listings...


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])
num_categories = len(label_encoder.classes_)

In [ ]:
print("Processing resume...")
resume_text = cleaned_resume
resume_embedding = model_st.encode([resume_text])[0]

Processing resume...


In [ ]:
def create_nn_matching_model(embedding_dim=384):
    """
    Create a neural network that learns to match resumes to jobs
    """
    resume_input = Input(shape=(embedding_dim,), name="resume_embedding")
    job_input = Input(shape=(embedding_dim,), name="job_embedding")

    resume_dense = Dense(256, activation='relu')(resume_input)
    resume_dropout = Dropout(0.3)(resume_dense)
    resume_dense2 = Dense(128, activation='relu')(resume_dropout)

    job_dense = Dense(256, activation='relu')(job_input)
    job_dropout = Dropout(0.3)(job_dense)
    job_dense2 = Dense(128, activation='relu')(job_dropout)


    combined = Concatenate()([resume_dense2, job_dense2])

    dense_combined = Dense(128, activation='relu')(combined)
    dropout_combined = Dropout(0.3)(dense_combined)
    dense_combined2 = Dense(64, activation='relu')(dropout_combined)


    output = Dense(1, activation='sigmoid', name="similarity_score")(dense_combined2)


    model = Model(inputs=[resume_input, job_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    return model


In [ ]:
def create_category_prediction_model(embedding_dim=384, num_categories=num_categories):


    resume_input = Input(shape=(embedding_dim,), name="resume_embedding")

    dense1 = Dense(256, activation='relu')(resume_input)
    dropout1 = Dropout(0.3)(dense1)
    dense2 = Dense(128, activation='relu')(dropout1)
    dropout2 = Dropout(0.3)(dense2)

    output = Dense(num_categories, activation='softmax', name="category_prediction")(dropout2)


    model = Model(inputs=resume_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    return model


In [ ]:
def prepare_training_data(job_embeddings, df, sample_size=10000):

    X_resume = []
    X_job = []
    y = []


    category_groups = df.groupby('category_encoded')
    categories = list(category_groups.groups.keys())


    pairs_created = 0
    while pairs_created < sample_size:

        is_positive = np.random.choice([True, False])

        if is_positive:

            category = np.random.choice(categories)

            jobs_in_category = category_groups.get_group(category)
            if len(jobs_in_category) < 2:
                continue


            job_indices = np.random.choice(jobs_in_category.index.values, 2, replace=False)


            X_resume.append(job_embeddings[job_indices[0]])
            X_job.append(job_embeddings[job_indices[1]])
            y.append(1)

        else:

            categories_sample = np.random.choice(categories, 2, replace=False)

            job1 = np.random.choice(category_groups.get_group(categories_sample[0]).index.values)
            job2 = np.random.choice(category_groups.get_group(categories_sample[1]).index.values)


            X_resume.append(job_embeddings[job1])
            X_job.append(job_embeddings[job2])
            y.append(0)

        pairs_created += 1

    return np.array(X_resume), np.array(X_job), np.array(y)



In [ ]:
def prepare_category_data(job_embeddings, df):

    X = job_embeddings
    y = df['category_encoded'].values

    return X, y


In [ ]:
print("Preparing training data...")
X_resume, X_job, y_matching = prepare_training_data(job_embeddings, df)

Preparing training data...


In [ ]:
X_cat, y_cat = prepare_category_data(job_embeddings, df)


In [ ]:
X_resume_train, X_resume_val, X_job_train, X_job_val, y_matching_train, y_matching_val = train_test_split(
    X_resume, X_job, y_matching, test_size=0.2, random_state=42)

In [ ]:
X_cat_train, X_cat_val, y_cat_train, y_cat_val = train_test_split(
    X_cat, y_cat, test_size=0.2, stratify=y_cat, random_state=42)


In [ ]:
print("Training job matching model...")
matching_model = create_nn_matching_model()
matching_model.fit(
    [X_resume_train, X_job_train], y_matching_train,
    validation_data=([X_resume_val, X_job_val], y_matching_val),
    epochs=10, batch_size=32, verbose=1
)

Training job matching model...
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.5270 - loss: 0.6746 - val_accuracy: 0.8090 - val_loss: 0.3883
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7981 - loss: 0.4058 - val_accuracy: 0.8610 - val_loss: 0.3363
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8767 - loss: 0.2832 - val_accuracy: 0.9345 - val_loss: 0.1719
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9258 - loss: 0.1910 - val_accuracy: 0.9390 - val_loss: 0.1598
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9366 - loss: 0.1611 - val_accuracy: 0.9395 - val_loss: 0.1630
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9357 - loss: 0.1631 - val_accuracy: 0.9420 - val_loss: 0.1504
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9416 - loss: 0.1474 - val_accuracy: 0.9460 - val_loss: 0.1418
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9377

In [ ]:
print("Training category prediction model...")
category_model = create_category_prediction_model()
category_model.fit(
    X_cat_train, y_cat_train,
    validation_data=(X_cat_val, y_cat_val),
    epochs=10, batch_size=32, verbose=1
)

Training category prediction model...
Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4332 - loss: 1.5336 - val_accuracy: 0.7650 - val_loss: 1.0549
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8036 - loss: 0.8615 - val_accuracy: 0.8803 - val_loss: 0.4348
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9082 - loss: 0.3621 - val_accuracy: 0.9060 - val_loss: 0.2925
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9166 - loss: 0.2643 - val_accuracy: 0.9188 - val_loss: 0.2568
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9441 - loss: 0.1883 - val_accuracy: 0.9316 - val_loss: 0.2391
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9670 - loss: 0.1315 - val_accuracy: 0.9188 - val_loss: 0.2610
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9550 - loss: 0.1506 - val_accuracy: 0.9316 - val_loss: 0.2441
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9657 - loss: 0.

In [ ]:
def recommend_jobs_dl(resume_embedding, job_embeddings_df, matching_model, df, top_n=10):

    job_indices = job_embeddings_df.index.values
    job_embs = job_embeddings

    resume_embeddings = np.array([resume_embedding] * len(job_embs))


    similarity_scores = matching_model.predict([resume_embeddings, job_embs], verbose=0).flatten()


    job_scores = list(zip(job_indices, similarity_scores))


    job_scores.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = []
    for idx, score in job_scores[:top_n]:
        job_info = df.loc[idx]
        top_recommendations.append({
            'job_id': job_info['original_job_id'],
            'job_title': job_info['job_title'],
            'category': job_info['category'],
            'skills': job_info['job_skill_set'],
            'similarity_score': float(score)
        })

    return top_recommendations

In [ ]:
category_probs = category_model.predict(np.array([resume_embedding]), verbose=0)[0]
predicted_category_idx = np.argmax(category_probs)
predicted_category = label_encoder.inverse_transform([predicted_category_idx])[0]
category_confidence = category_probs[predicted_category_idx]


In [ ]:
print(f"\nPredicted Resume Category: {predicted_category} (Confidence: {category_confidence:.2f})")



Predicted Resume Category: business-development (Confidence: 0.78)


In [ ]:
print("\nFinding job matches using neural network...")
dl_recommendations = recommend_jobs_dl(resume_embedding, pd.DataFrame(job_embeddings), matching_model, df)
print("\n===== TOP JOB RECOMMENDATIONS (DEEP LEARNING) =====")
for i, rec in enumerate(dl_recommendations, 1):
    print(f"\n{i}. {rec['job_title']}")
    print(f"   Category: {rec['category']}")
    print(f"   Required Skills: {rec['skills']}")
    print(f"   Match Score: {rec['similarity_score']:.4f}")




Finding job matches using neural network...

===== TOP JOB RECOMMENDATIONS (DEEP LEARNING) =====

1. Sr. SAP Finance Consultant/Architect
   Category: finance
   Required Skills: Training Development, Communication, Collaboration, Problem Solving, Leadership, Interpersonal Skills, Trust Building, Coaching, Stakeholder Management, Adaptability, SAP FICO, S/4 HANA, Financial Reporting, Process Automation, Agile Methodologies, Business Requirements Documents (BRDs), Financial Analysis, Technical Solutions Design, Critical Analytics, Detail-Oriented
   Match Score: 0.8667

2. Senior Finance Manager, Reporting Innovation
   Category: finance
   Required Skills: financial reporting, FP&A processes, MS Office, change management, leadership, analytical skills, communication, influence, problem solving, adaptability, collaboration, intellectual curiosity, data analytics, SAP, SOX compliance, RPA, NLG, AI, ML, multi-ERP systems, innovation
   Match Score: 0.8632

3. Lead Software Engineer - Fin

In [ ]:
def evaluate_dl_models(matching_model, category_model, test_df, label_encoder):

    X_test = model_st.encode(test_df['combined_text'].tolist())
    y_test = test_df['category_encoded'].values

    y_pred_probs = category_model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'y_pred': y_pred,
        'y_test': y_test
    }

from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


evaluation_results = evaluate_dl_models(matching_model, category_model, test_df, label_encoder)

evaluation_results = evaluate_dl_models(matching_model, category_model, test_df, label_encoder)

print("\n===== MODEL EVALUATION =====")
print(f"- Accuracy: {evaluation_results['accuracy']:.4f}")
print(f"- Precision: {evaluation_results['precision']:.4f}")
print(f"- Recall: {evaluation_results['recall']:.4f}")
print(f"- F1 Score: {evaluation_results['f1']:.4f}")


y_pred_labels = label_encoder.inverse_transform(evaluation_results['y_pred'])
y_true_labels = label_encoder.inverse_transform(evaluation_results['y_test'])





===== MODEL EVALUATION =====
- Accuracy: 0.9829
- Precision: 0.9832
- Recall: 0.9829
- F1 Score: 0.9829


In [ ]:
matching_model.export('/content/job_matching_model')
category_model.export('/content/job_category_model')

print("\nModels saved to '/content/job_matching_model' and '/content/job_category_model'")


In [ ]:
import shutil

# Zip the job matching model
shutil.make_archive('/content/job_matching_model', 'zip', '/content/job_matching_model')

# Zip the job category model
shutil.make_archive('/content/job_category_model', 'zip', '/content/job_category_model')


In [ ]:
from google.colab import files

# Download both zipped models
files.download('/content/job_matching_model.zip')
files.download('/content/job_category_model.zip')


#SBERT FINE TUNING

In [ ]:
!pip install -U sentence-transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.4.1
    Uninstalling sentence-transformers-3.4.1:
      Successfully uninstalled sentence-transformers-3.4.1


In [ ]:
from datasets import load_dataset
import pandas as pd



# We'll use only the 'train' split

df = pd.read_csv("/content/cleaned_job_dataset.csv")
df = df[['job_title', 'job_description']].dropna().reset_index(drop=True)


In [ ]:
# Load dataset
dataset = pd.read_csv("/content/cleaned_job_dataset.csv")
print(dataset.head(10))

       job_id category                                      job_title  \
0  3902668440       hr                   Sr Human Resource Generalist   
1  3905823748       hr                        Human Resources Manager   
2  3905854799       hr                    Director of Human Resources   
3  3905834061       hr                  Chief Human Resources Officer   
4  3906250451       hr       Human Resources Generalist (Hybrid Role)   
5  3901389277       hr                        Human Resources Manager   
6  3902348043       hr                     Human Resources Generalist   
7  3906258424       hr  Human Resources Information System Specialist   
8  3891070825       hr                     Human Resources Generalist   
9  3901949090       hr                Human Resources Project Manager   

                                     job_description  \
0  summary sr hr generalist provides hr expertise...   
1  part stellar team ysb manager human resource j...   
2  client thriving organizat

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

df['job_description'] = df['job_description'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sentence_transformers import InputExample
import random

triplets = []
grouped = df.groupby("job_title")

for idx, row in df.iterrows():
    anchor = row['job_description']
    job_title = row['job_title']

    pos_pool = grouped.get_group(job_title).drop(index=idx, errors='ignore')
    if pos_pool.empty:
        continue
    positive = pos_pool.sample(1).iloc[0]['job_description']

    neg_pool = df[df['job_title'] != job_title]
    negative = neg_pool.sample(1).iloc[0]['job_description']

    triplets.append(InputExample(texts=[anchor, positive, negative]))

print(f"Generated {len(triplets)} triplets.")


Generated 542 triplets.


In [ ]:
from sentence_transformers import InputExample
import random

triplets = []
grouped = df.groupby("job_title")

for idx, row in df.iterrows():
    anchor = row['job_description']
    job_title = row['job_title']

    pos_pool = grouped.get_group(job_title).drop(index=idx, errors='ignore')
    if pos_pool.empty:
        continue
    positive = pos_pool.sample(1).iloc[0]['job_description']

    neg_pool = df[df['job_title'] != job_title]
    negative = neg_pool.sample(1).iloc[0]['job_description']

    triplets.append(InputExample(texts=[anchor, positive, negative]))

print(f"Generated {len(triplets)} triplets.")


Generated 542 triplets.


In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses

model = SentenceTransformer('all-MiniLM-L6-v2')
train_dataloader = DataLoader(triplets, shuffle=True, batch_size=8)
train_loss = losses.TripletLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=2,
    warmup_steps=100,
    show_progress_bar=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: venu20priya (venu20priya-shiv-nadar-university-chennai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
model.save("fine-tuned-sbert-job-skill")
print("Model saved!")


Model saved!


In [ ]:
import shutil

shutil.make_archive("fine-tuned-sbert-job-skill", 'zip', "fine-tuned-sbert-job-skill")
print("Model zipped!")


Model zipped!


In [ ]:
from google.colab import files

files.download("fine-tuned-sbert-job-skill.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sentence_transformers import SentenceTransformer

# Load your fine-tuned model
model = SentenceTransformer("fine-tuned-sbert-job-skill")


In [ ]:
# Generate embeddings for all job entries
df=pd.read_csv("/content/final_combined_jobs.csv")
job_embeddings = model.encode(df['combined_text'].tolist(), show_progress_bar=True, convert_to_tensor=True)


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:
resume_embedding = model.encode([cleaned_resume], convert_to_tensor=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

# Encode categories
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])
num_categories = len(label_encoder.classes_)

def create_nn_matching_model(embedding_dim=384):
    resume_input = Input(shape=(embedding_dim,), name="resume_embedding")
    job_input = Input(shape=(embedding_dim,), name="job_embedding")

    resume_dense = Dense(256, activation='relu')(resume_input)
    resume_dropout = Dropout(0.3)(resume_dense)
    resume_dense2 = Dense(128, activation='relu')(resume_dropout)

    job_dense = Dense(256, activation='relu')(job_input)
    job_dropout = Dropout(0.3)(job_dense)
    job_dense2 = Dense(128, activation='relu')(job_dropout)

    combined = Concatenate()([resume_dense2, job_dense2])
    dense_combined = Dense(128, activation='relu')(combined)
    dropout_combined = Dropout(0.3)(dense_combined)
    dense_combined2 = Dense(64, activation='relu')(dropout_combined)

    output = Dense(1, activation='sigmoid', name="similarity_score")(dense_combined2)

    model = Model(inputs=[resume_input, job_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

def create_category_prediction_model(embedding_dim=384, num_categories=num_categories):
    resume_input = Input(shape=(embedding_dim,), name="resume_embedding")
    dense1 = Dense(256, activation='relu')(resume_input)
    dropout1 = Dropout(0.3)(dense1)
    dense2 = Dense(128, activation='relu')(dropout1)
    dropout2 = Dropout(0.3)(dense2)

    output = Dense(num_categories, activation='softmax', name="category_prediction")(dropout2)

    model = Model(inputs=resume_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def prepare_training_data(job_embeddings, df, sample_size=10000):
    X_resume, X_job, y = [], [], []
    category_groups = df.groupby('category_encoded')
    categories = list(category_groups.groups.keys())

    pairs_created = 0
    while pairs_created < sample_size:
        is_positive = np.random.choice([True, False])
        if is_positive:
            category = np.random.choice(categories)
            jobs_in_category = category_groups.get_group(category)
            if len(jobs_in_category) < 2:
                continue
            job_indices = np.random.choice(jobs_in_category.index.values, 2, replace=False)
            X_resume.append(job_embeddings[job_indices[0]])
            X_job.append(job_embeddings[job_indices[1]])
            y.append(1)
        else:
            categories_sample = np.random.choice(categories, 2, replace=False)
            job1 = np.random.choice(category_groups.get_group(categories_sample[0]).index.values)
            job2 = np.random.choice(category_groups.get_group(categories_sample[1]).index.values)
            X_resume.append(job_embeddings[job1])
            X_job.append(job_embeddings[job2])
            y.append(0)
        pairs_created += 1
    return np.array(X_resume), np.array(X_job), np.array(y)

def prepare_category_data(job_embeddings, df):
    X = job_embeddings
    y = df['category_encoded'].values
    return X, y

# Prepare data
print("Preparing training data...")
X_resume, X_job, y_matching = prepare_training_data(job_embeddings, df)
X_cat, y_cat = prepare_category_data(job_embeddings, df)

# Split
X_resume_train, X_resume_val, X_job_train, X_job_val, y_matching_train, y_matching_val = train_test_split(
    X_resume, X_job, y_matching, test_size=0.2, random_state=42)
X_cat_train, X_cat_val, y_cat_train, y_cat_val = train_test_split(
    X_cat, y_cat, test_size=0.2, stratify=y_cat, random_state=42)

# Train matching model
print("Training job matching model...")
matching_model = create_nn_matching_model()
matching_model.fit(
    [X_resume_train, X_job_train], y_matching_train,
    validation_data=([X_resume_val, X_job_val], y_matching_val),
    epochs=10, batch_size=32, verbose=1
)

# Train category model
print("Training category prediction model...")
category_model = create_category_prediction_model()
category_model.fit(
    X_cat_train, y_cat_train,
    validation_data=(X_cat_val, y_cat_val),
    epochs=10, batch_size=32, verbose=1
)

# Recommendation function
def recommend_jobs_dl(resume_embedding, job_embeddings_df, matching_model, df, top_n=10):
    job_indices = job_embeddings_df.index.values
    job_embs = job_embeddings_df.values  # (N, 384)

    resume_embedding = resume_embedding.reshape(1, -1)  # Ensure shape (1, 384)
    resume_embeddings = np.repeat(resume_embedding, len(job_embs), axis=0)  # (N, 384)

    similarity_scores = matching_model.predict([resume_embeddings, job_embs], verbose=0).flatten()
    job_scores = list(zip(job_indices, similarity_scores))
    job_scores.sort(key=lambda x: x[1], reverse=True)

    top_recommendations = []
    for idx, score in job_scores[:top_n]:
        job_info = df.loc[idx]
        top_recommendations.append({
            'job_id': job_info['job_id'],
            'job_title': job_info['job_title'],
            'category': job_info['category'],
            'skills': job_info['job_skill_set'],
            'similarity_score': float(score)
        })
    return top_recommendations

# Run a sample prediction
resume_embedding = resume_embedding.reshape(1, -1)  # Ensure it's the correct shape
category_probs = category_model.predict(resume_embedding, verbose=0)[0]
predicted_category_idx = np.argmax(category_probs)
predicted_category = label_encoder.inverse_transform([predicted_category_idx])[0]
category_confidence = category_probs[predicted_category_idx]

print(f"\nPredicted Category: {predicted_category} ({category_confidence:.2f} confidence)")
print("\nFinding job matches using neural network...")

job_embeddings_df = pd.DataFrame(job_embeddings)  # Wrap job_embeddings in DataFrame
dl_recommendations = recommend_jobs_dl(resume_embedding, job_embeddings_df, matching_model, df)

print("\n===== TOP JOB RECOMMENDATIONS (DEEP LEARNING) =====")
for i, rec in enumerate(dl_recommendations, 1):
    print(f"\n{i}. {rec['job_title']}")
    print(f"   Category: {rec['category']}")
    print(f"   Required Skills: {rec['skills']}")
    print(f"   Match Score: {rec['similarity_score']:.4f}")

# Prepare test_df for evaluation
test_df = df.sample(frac=0.2, random_state=42).copy()
test_df['combined_text'] = test_df['job_title'] + " " + test_df['job_description'] + " " + test_df['job_skill_set']
test_df['category_encoded'] = label_encoder.transform(test_df['category'])

# Evaluation
def evaluate_dl_models(category_model, test_df, label_encoder):
    X_test = model.encode(test_df['combined_text'].tolist())
    y_test = test_df['category_encoded'].values

    y_pred_probs = category_model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'y_pred': y_pred,
        'y_test': y_test
    }

evaluation_results = evaluate_dl_models(category_model, test_df, label_encoder)

print("\n===== MODEL EVALUATION =====")
print(f"- Accuracy: {evaluation_results['accuracy']:.4f}")
print(f"- Precision: {evaluation_results['precision']:.4f}")
print(f"- Recall: {evaluation_results['recall']:.4f}")
print(f"- F1 Score: {evaluation_results['f1']:.4f}")


Preparing training data...
Training job matching model...
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.8054 - loss: 0.4212 - val_accuracy: 0.9225 - val_loss: 0.2120
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.9369 - loss: 0.1857 - val_accuracy: 0.9705 - val_loss: 0.1251
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9610 - loss: 0.1236 - val_accuracy: 0.9775 - val_loss: 0.0998
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9742 - loss: 0.0973 - val_accuracy: 0.9725 - val_loss: 0.0960
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9707 - loss: 0.0907 - val_accuracy: 0.9780 - val_loss: 0.0736
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9784 - loss: 0.0661 - val_accuracy: 0.9805 - val_loss: 0.0787
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9826 - loss: 0.0622 - val_accuracy: 0.9815 - val_loss: 0.0679
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [ ]:
# Save the matching model
matching_model.save("matching_model.h5")

# Save the category model
category_model.save("category_model.h5")

# Your sentence transformer model should already be saved
# (from your previous code: model.save("fine-tuned-sbert-job-skill"))

In [ ]:
# Step 1: Save both models (SavedModel format)
matching_model.save("/content/job_matching_model")
category_model.save("/content/job_category_model")

print("Models saved to disk.")

# Step 2: Zip the folders
import shutil

shutil.make_archive("/content/job_matching_model", 'zip', "/content/job_matching_model")
shutil.make_archive("/content/job_category_model", 'zip', "/content/job_category_model")

print("Models zipped successfully.")

# Step 3: Download the zipped models to your PC
from google.colab import files

files.download("/content/job_matching_model.zip")
files.download("/content/job_category_model.zip")


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/content/job_matching_model.

In [ ]:
# Save models and label encoder
matching_model.save('job_matching_model.h5')
category_model.save('category_prediction_model.h5')

# Save label encoder
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Create zip file
!zip models.zip job_matching_model.h5 category_prediction_model.h5 label_encoder.pkl

# Download the zip file
from google.colab import files
files.download('models.zip')

  adding: job_matching_model.h5 (deflated 8%)
  adding: category_prediction_model.h5 (deflated 9%)
  adding: label_encoder.pkl (deflated 19%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>